# Define Functions and Run Initial Query

In [3]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 10

def read_query_file(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df

df = run_query(read_query_file('SQL\SamplingTest\KeyMetrics_ForGroupSelection.SQL'))
df.head()

DatabaseError: Execution failed on sql 'WITH
DateSelector As (
    /*
    Builds a table with min and max dates.
    Kind of overengineered but it prevents dates
    getting changed in one place but not another.
    */
    SELECT *
    FROM
        ( VALUES (CAST('2023-06-01' AS DATE), CAST(CURRENT_DATE AS DATE)) )
        as t ("StartDate","EndDate")
),

EligibleEmployees AS(
    /*Finds currently active
     Employees in the orlando office and their tenure group */
SELECT
    CAST(eh.expert_id AS BIGINT) as "Employee",
    eh.tenure_group as "TenureGroup"
FROM
    hive.care.l3_asurion_whole_home_expert_hierarchy eh
WHERE 1=1
    AND UPPER(eh.business_unit) = 'PSS'
    AND eh.location = 'flor'
    AND (SELECT MAX(EndDate) FROM DateSelector)
        BETWEEN eh.eff_start_dt and eh.eff_end_dt
),

EfficiencyMetrics AS(
    /*
    Agregates total hous worked and total sessions and finds total resolution time for crt
    */
    SELECT 
        CAST(sfd.emplid AS BIGINT) as "Employee",
     

In [ ]:
df['CXP_Prob_Count'].value_counts()

# Build out the metrics from the Query

In [ ]:
df['SP100'] = df['Total_Accepted']/df['Total_Eligible']
df['CXP_Score']=df['CXP_Prob_Sum']/df['CXP_Prob_Count']
df['Sessions Per Hour'] = df['Sessions']/df['HoursWorked']
df['CRT']= df['Total_Resolution_Time']/df['Sessions']
df['Searches Per Session']= df['Helix_Searches']/df['Sessions']
df['%Sessions With Search']= df['Helix_Sessions']/df['Sessions']
df=df.join(pd.get_dummies(df['TenureGroup']))
employees_df = df[['Employee','TenureGroup']].copy().drop_duplicates(subset='Employee')
display(employees_df)
display(df.head())

# Select Sample Size by Tenure Group
Using the current number of experts find the number of samples needed in each group to be representative of the general population at Orlando

In [ ]:
SAMPLE_SIZE = 30
sample_size_df=pd.pivot_table(df.copy(),index='TenureGroup',values='Employee',aggfunc=lambda x: len(x.unique()),margins=True,)
sample_size_df['Total Employees']=sample_size_df['Employee']
del sample_size_df['Employee']
sample_size_df['% of Employees'] = sample_size_df['Total Employees']/sample_size_df['Total Employees'].loc['All']
sample_size_df['n Employees'] = (sample_size_df['% of Employees']* SAMPLE_SIZE).round()

sample_size_df=sample_size_df.iloc[:-1]
display(sample_size_df.sum())
display(sample_size_df)

del SAMPLE_SIZE

# Test
1. Creates representative samples
2. Tests those samples to see if they have a meaningful effect on the metric
    1. Runs an OLS on the sample to see if they have high p val for the metric. If they have a low value for all metrics they are saved as a good possible sample.

In [ ]:
P_VAL_FLOOR = .5
SAMPLES = 10000

SelectionMetrics = ['CRT','%Sessions With Search','SP100','CXP_Score']

results_df = pd.DataFrame(columns=['Members']+SelectionMetrics)

# Build test dataframe dft and get groups setup
groups = sample_size_df.index.to_list()
dft = df[['Employee']+SelectionMetrics+groups].copy()
dft['Constant']=1
dft = dft.replace({np.inf : np.nan,np.inf : np.nan})
dft= dft.dropna()

# Make a loop to run this a bunch
for run in range(1,SAMPLES):
    # creates a list for the sampled employes
    # then adds the correct number of samples from each group to it
    sample_group = []
    for group in groups:
        sample_df = employees_df[employees_df['TenureGroup']==group]
        sample_group= sample_group+ sample_df['Employee'].sample(
            n=int(sample_size_df.loc[group]['n Employees'])
            ,replace=False
            ,random_state=run
            ).tolist()
    del group
    
    # Flag the employees in the group
    dft['TestSample']=(dft['Employee'].isin(sample_group)).astype(int)
    
    # Make sure the p vals are high enough to assume no statistical significance
    # Also added a lower and upper bound to make sure it could be positive or negative
    p_val_list = []
    for y in SelectionMetrics:
        model = sm.OLS(endog=dft[y].copy(),exog=dft.copy()[groups+['TestSample','Constant']]).fit().summary2().tables[1]
        p_val = model['P>|t|'].loc['TestSample']
        lower = model['[0.025'].loc['TestSample']
        upper = model['0.975]'].loc['TestSample']
        if p_val >= P_VAL_FLOOR and lower<0 and upper>0:
            p_val_list.append(p_val)
        del p_val, y
    # add sample group to results list if the results were not significant
    if len(p_val_list)==len(SelectionMetrics):
        current_result = dict(zip(['Members']+SelectionMetrics,[[sample_group]]+p_val_list))
        results_df=pd.concat([results_df,pd.DataFrame(current_result,index=[run])])
    del sample_group, p_val_list, dft['TestSample'], sample_df

del dft, SAMPLES, P_VAL_FLOOR, run, groups, 

results_df
# results_df.to_excel('Unbiased Sample Selections.xlsx')

# Find and display subsets that do not have any duplicate members

In [ ]:
results_df = pd.read_excel('Unbiased Sample Selections.xlsx')
# SelectionMetrics = ['CRT','%Sessions With Search','SP100','CXP_Score']
results_df['TotalPVal']=0
for metric in SelectionMetrics:
    results_df['TotalPVal'] = results_df['TotalPVal']+results_df[metric]
results_df['AvgPVal']= results_df['TotalPVal']/len(SelectionMetrics)

del metric
for row1 in range(len(results_df['Members'])):
    for row2 in range(1,len(results_df['Members'])):
        duplicates = 0
        for item in results_df['Members'].iloc[row1]:
            if item in results_df['Members'].iloc[row2]: 
                duplicates=duplicates+1
                display(item)
        if duplicates == 0:
            display(pd.concat([results_df.iloc[[row1]],results_df.iloc[[row2]]]))
del row1, row2, duplicates, item

In [ ]:
results_df.to_excel('Unbiased Sample Selections.xlsx')

# Changes
1. Drop Sessions Per Hour
2. Get SP100 and CXP from Brian Vickers
3. Check on Helx Search Ravi